In [1]:
# Makes bad pixel mask for AB Aur data

# Created by E.S., 2018 October

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
#import sys
import glob, os
%matplotlib qt

In [3]:
# read in sample frame

stem = '/Users/bandari/Documents/git.repos/dirac/vtp_scripts_data/nirao_09_dark_current/data/'
file_name = 'tests_junk_13may/pos1_selected_cold_target_not_cold/10sec.fits'

hdul = fits.open(stem + file_name)
sample_frame = hdul[0].data

# make bad pix canvas (0: good; 1: bad)
bad_pix_frame = np.zeros(sample_frame.shape, dtype=int)

In [4]:
# mask overscan region
bad_pix_frame[:,0:4] = 1
bad_pix_frame[:,-4:] = 1
bad_pix_frame[0:4,:] = 1
bad_pix_frame[-4:,:] = 1

In [12]:
plt.imshow(bad_pix_frame)
plt.show()

In [5]:
fits.writeto(stem + 'junk_bad_pix.fits', bad_pix_frame, overwrite=True)


In [6]:
np.nansum(bad_pix_frame)

16320

In [16]:
print(stem + 'junk_bad_pix.fits')

/Users/bandari/Documents/git.repos/dirac/data/nirao_09_dark_current/tests_junk_13may/pos1_selected_cold_target_not_cold/junk_bad_pix.fits


In [3]:
#stem = '/Users/bandari/Documents/git.repos/hd_141569/overview_data/'
#stem_read = '/Volumes/One Touch/abaur_data/00_slope_frames/' # read the slope frames
#stem_write = '/Volumes/One Touch/abaur_data/calib_frames/'

dir_calib_frames = '/Volumes/One Touch/abaur_data/calib_frames/'
dir_dark_subted = '/Volumes/One Touch/abaur_data/01_dark_subtracted/'

In [4]:
# to check for variation in pixel responses

# darks are frames 14288 to 14437
nums_of_interest = np.arange(14288,14438,1)

cube_dark_slopes = np.zeros((len(nums_of_interest),1024,2048))

# put individual dark slope frames into cube
# bad pixels are where variations are highest 
for i in range(0,len(nums_of_interest)):

    file_basename = 'lm_190108_0' + str(int(nums_of_interest[i])) + '.fits'
    #file_basename = 'lm_190108_0' + str(13000) + '.fits'

    print(file_basename)
                                         
    hdul = fits.open(dir_dark_subted + file_basename)

    frame_this = hdul[0].data

    # one entire channel is bad; mask this
    frame_this[:,14*64:15*64] = np.nan

    hdul = fits.open(dir_dark_subted + file_basename)            
    cube_dark_slopes[i,:,:] = frame_this
                                         
    

lm_190108_014288.fits
lm_190108_014289.fits
lm_190108_014290.fits
lm_190108_014291.fits
lm_190108_014292.fits
lm_190108_014293.fits
lm_190108_014294.fits
lm_190108_014295.fits
lm_190108_014296.fits
lm_190108_014297.fits
lm_190108_014298.fits
lm_190108_014299.fits
lm_190108_014300.fits
lm_190108_014301.fits
lm_190108_014302.fits
lm_190108_014303.fits
lm_190108_014304.fits
lm_190108_014305.fits
lm_190108_014306.fits
lm_190108_014307.fits
lm_190108_014308.fits
lm_190108_014309.fits
lm_190108_014310.fits
lm_190108_014311.fits
lm_190108_014312.fits
lm_190108_014313.fits
lm_190108_014314.fits
lm_190108_014315.fits
lm_190108_014316.fits
lm_190108_014317.fits
lm_190108_014318.fits
lm_190108_014319.fits
lm_190108_014320.fits
lm_190108_014321.fits
lm_190108_014322.fits
lm_190108_014323.fits
lm_190108_014324.fits
lm_190108_014325.fits
lm_190108_014326.fits
lm_190108_014327.fits
lm_190108_014328.fits
lm_190108_014329.fits
lm_190108_014330.fits
lm_190108_014331.fits
lm_190108_014332.fits
lm_190108_

In [27]:
#fits.writeto('junk.fits', cube_dark_slopes, overwrite=True)

In [5]:
# find variation in pixels
sorted_stds = np.sort(np.ndarray.flatten(np.std(cube_dark_slopes, axis=0)))

std_frame = np.std(cube_dark_slopes, axis=0)

#plt.plot(sorted_stds)
#plt.show()


In [6]:
plt.imshow(std_frame)
plt.show()

In [7]:
# read in master dark
hdul_master = fits.open(dir_calib_frames + 'master_dark_slopes.fits') 
master_dark = hdul[0].data

# one entire channel is bad; mask this
master_dark[:,14*64:15*64] = np.nan

sorted_vals = np.sort(np.ndarray.flatten(master_dark))

In [8]:
# show pixels based on criterion

lower_limit = -11
upper_limit = 10

test_mask = np.zeros(np.shape(master_dark))
#test_mask[frame_this > lower_limit] = 1
test_mask[np.logical_and(master_dark > lower_limit, master_dark < upper_limit)] = 1

plt.clf()
plt.imshow(test_mask, origin='lower')
plt.show()


In [9]:
# make the bad pixel mask
# first step is to assign 'True' to every pixel which is bad, and write as array of ints

# Criterion 1: where are hot and cold pixels?
cold_limit = -11
hot_limit = 10
substrate_mask = np.full(np.shape(master_dark), False)
substrate_mask[np.logical_or(master_dark < cold_limit, master_dark > hot_limit)] = True

# Criterion 2: Mask an entire bad column
substrate_mask[:,14*64:15*64] = True

# Criterion 3: Where is variation highest?
noise_limit = 7 # 7
dead_limit = 1.5
substrate_mask[np.logical_or(std_frame < dead_limit, std_frame > noise_limit)] = True

# write to file as ints
# 1: bad
# 0: good
fits.writeto('junk.fits', substrate_mask.astype(int), overwrite=True)

In [83]:
from astropy.io import fits
# take first version of bad pixel mask, read in an illuminated frame, and flag dark pixels (dust)

file_path_badpix_v1 = '/Volumes/One Touch/abaur_data/calib_frames/master_bad_pix.fits'
hdul_bad_v1 = fits.open(file_path_badpix_v1)
bad_v1 = hdul_bad_v1[0].data
hdul_bad_v1.close()

# read in 4 illuminated frames with the Lcont4 filter; those with consistent dark pixels are likely dust
# (Lcont4 filter much dustier than the H2O-Ice2 filter)
file_path_illuminated_1 = '/Volumes/One Touch/abaur_data/03_ramp_removed/lm_190108_001424.fits'
file_path_illuminated_2 = '/Volumes/One Touch/abaur_data/03_ramp_removed/lm_190108_003533.fits'
file_path_illuminated_3 = '/Volumes/One Touch/abaur_data/03_ramp_removed/lm_190108_007076.fits'
file_path_illuminated_4 = '/Volumes/One Touch/abaur_data/03_ramp_removed/lm_190108_011436.fits'
hdul_illum_1 = fits.open(file_path_illuminated_1)
hdul_illum_2 = fits.open(file_path_illuminated_2)
hdul_illum_3 = fits.open(file_path_illuminated_3)
hdul_illum_4 = fits.open(file_path_illuminated_4)
illum_1 = hdul_illum_1[0].data
illum_2 = hdul_illum_2[0].data
illum_3 = hdul_illum_3[0].data
illum_4 = hdul_illum_4[0].data
hdul_illum_1.close()
hdul_illum_2.close()
hdul_illum_3.close()
hdul_illum_4.close()


In [79]:
# flag dark pixels

dividing_line_1 = 10
dividing_line_2 = -3
dividing_line_3 = -2
dividing_line_4 = -1

'''
illum_1[illum_1<dividing_line_1] = 1
illum_1[illum_1>dividing_line_1] = 0

illum_2[illum_2<dividing_line_2] = 1
illum_2[illum_2>dividing_line_2] = 0

illum_3[illum_3<dividing_line_3] = 1
illum_3[illum_3>dividing_line_3] = 0

illum_4[illum_4<dividing_line_4] = 1
illum_4[illum_4>dividing_line_4] = 0
'''

'\nillum_1[illum_1<dividing_line_1] = 1\nillum_1[illum_1>dividing_line_1] = 0\n\nillum_2[illum_2<dividing_line_2] = 1\nillum_2[illum_2>dividing_line_2] = 0\n\nillum_3[illum_3<dividing_line_3] = 1\nillum_3[illum_3>dividing_line_3] = 0\n\nillum_4[illum_4<dividing_line_4] = 1\nillum_4[illum_4>dividing_line_4] = 0\n'

In [40]:
# Write illum_1 as a FITS file
fits.writeto('junk_illum_1.fits', illum_1, overwrite=True)

# Write illum_2 as a FITS file
fits.writeto('junk_illum_2.fits', illum_2, overwrite=True)

# Write illum_3 as a FITS file
fits.writeto('junk_illum_3.fits', illum_3, overwrite=True)

# Write illum_4 as a FITS file
fits.writeto('junk_illum_4.fits', illum_4, overwrite=True)


In [34]:
illum_sum = np.add(illum_1, np.add(illum_2, np.add(illum_3, illum_4)))


In [38]:
print(np.nanmin(illum_sum))

0.0


In [60]:
#plt.imshow(illum_1, origin='lower')
#illum_2[illum_2<-4] = 1
#illum_2[illum_2>-4] = np.nan
plt.imshow(illum_2, origin='lower', vmin=-20, vmax=20)
plt.show()

In [63]:
plt.imshow(illum_2<-4, origin='lower')
plt.show()

In [89]:
# where condition is true in all four frames

dividing_line_1 = 10
dividing_line_2 = -3
dividing_line_3 = -2
dividing_line_4 = -1

frame_all_true = np.logical_and(np.logical_and(illum_1<dividing_line_1,illum_2<dividing_line_2),
                        np.logical_and(illum_3<dividing_line_3,illum_4<dividing_line_4))

#frame_all_true = np.logical_and(illum_1<dividing_line_1,illum_2<dividing_line_2)
#frame_all_true = np.logical_and(illum_2<dividing_line_2,np.logical_and(illum_3<dividing_line_3,illum_4<dividing_line_4))

plt.imshow(frame_all_true, origin='lower')
plt.show()

In [91]:
# Write frame_all_true as a FITS file
fits.writeto('junk_frame_all_true.fits', frame_all_true.astype(int), overwrite=True)


In [114]:
from astropy.io import fits
# Read in the bad pixel mask, and the dust mask, and combine them
file_name_badpix = '/Volumes/One Touch/abaur_data/calib_frames/master_bad_pix.fits'
file_name_dust = '/Volumes/One Touch/abaur_data/calib_frames/master_dust_frame.fits'
hdul_badpix = fits.open(file_name_badpix)
hdul_dust = fits.open(file_name_dust)

# 0: good; 1: bad
bad_sum = np.add(hdul_badpix[0].data, hdul_dust[0].data)
bad_sum[bad_sum > 0] = 1

# Write bad_pix + dust to file
fits.writeto('junk.fits', bad_sum, overwrite=True)



In [16]:
# read in two pairs of frames (one pair for each nod position) and flag the pixels that 
# are hot in both from each nod

frame_up_1 = fits.open('./hot_4473.fits')[0].data
frame_up_2 = fits.open('./hot_4580.fits')[0].data
frame_down_1 = fits.open('./hot_7976.fits')[0].data
frame_down_2 = fits.open('./hot_8126.fits')[0].data


hot_up = np.logical_and(frame_up_1 == 1, frame_up_2 == 1)
hot_down = np.logical_and(frame_down_1 == 1, frame_down_2 == 1)

hot_up_and_down = np.logical_or(hot_up,hot_down)


In [9]:
np.logical_and(frame_up_1 == 1, frame_up_2 == 1)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [17]:
plt.imshow(hot_up_and_down)
plt.show()

In [14]:
np.sum(hot_up)

2148

In [20]:
from astropy.io import fits
# Read in a bad pixel mask and add some more hot pixels
file_name_badpix = '/Volumes/One Touch/abaur_data/calib_frames/master_bad_pix_plus_dust.fits'
hdul_badpix = fits.open(file_name_badpix)

# 0: good; 1: bad
bad_sum = np.add(hdul_badpix[0].data, hot_up_and_down)
bad_sum[bad_sum > 0] = 1

# Write bad_pix + dust to file
fits.writeto('junk.fits', bad_sum, overwrite=True)

#fits.writeto('junk2.fits', hot_up_and_down.astype(int), overwrite=True)



In [108]:
np.shape(np.add(hdul_badpix[0].data, hdul_dust[0].data))

(1024, 2048)

In [98]:
type(hdul_dust[0].data[0,0])

numpy.int64

In [113]:
plt.imshow(bad_sum, origin='lower')
plt.show()

In [6]:
# cast to ints
#cube_flat_slopes = cube_flat_slopes.astype(int)

# write mean to file (note that the mean operation makes the data go from its to floats)
master_dark = np.mean(cube_flat_slopes, axis=0)

fits.writeto(stem_write + 'junk_master_dark.fits', master_dark, overwrite=True)